In [2]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

In [4]:
!pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.16.56 botocore==1.19.56

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.5 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.3.1 which is incompatible.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.2 which is incompatible.
sagemaker 2.184.0 requires boto3<2.0,>=1.26.131, but you have boto3 1.16.56 which is incompatible

In [5]:
!pip install --disable-pip-version-check -q sagemaker
!pip install --disable-pip-version-check -q smdebug==1.0.1
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.18.216 requires botocore==1.19.56, but you have botocore 1.31.57 which is incompatible.
awscli 1.18.216 requires PyYAML<5.4,>=3.10; python_version != "3.4", but you have pyyaml 6.0.1 which is incompatible.
awscli 1.18.216 requires s3transfer<0.4.0,>=0.3.0, but you have s3transfer 0.7.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.5 requires jupyter-client<8,>=5.3.4, but you have j

In [20]:
!python --version
!pip install --disable-pip-version-check -q matplotlib
!pip install --disable-pip-version-check -q seaborn

Python 3.10.6


In [21]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

s3 = boto3.Session().client(service_name="s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [22]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'RM2M1FZ865AT73ME', 'HostId': '6+bYUtgGwnPD2/L5ucz8KScnYKL9WZBNMxof5ieBkuPA+zkIHYSoOg1wjFQy83943jkC14jWvHs=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '6+bYUtgGwnPD2/L5ucz8KScnYKL9WZBNMxof5ieBkuPA+zkIHYSoOg1wjFQy83943jkC14jWvHs=', 'x-amz-request-id': 'RM2M1FZ865AT73ME', 'date': 'Thu, 28 Sep 2023 21:49:09 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


In [41]:
bucket_name = 'aai-540-group-4-dataset'

bucket_path = "s3://{}/".format(bucket_name)
print(bucket_name)

!aws s3 ls "s3://aai-540-group-4-dataset/"
print('---------')

aai-540-group-3-dataset
2023-09-28 21:30:35    4370528 dev-v2.0.json
2023-09-28 21:30:32   42123633 train-v2.0.json
---------


In [40]:
# Get caller identity
sts = boto3.client('sts')
response = sts.get_caller_identity()

# Output the account ID, user ID, and ARN
account_id = response['Account']
user_id = response['UserId']
arn = response['Arn']

print(f"Account ID: {account_id}")
print(f"User ID: {user_id}")
print(f"ARN: {arn}")

Account ID: 670204015151
User ID: AROAZYC2FWIX4P7SAG7YN:SageMaker
ARN: arn:aws:sts::670204015151:assumed-role/LabRole/SageMaker


In [44]:
import json
import pandas as pd
from pandas.io.json import json_normalize  # for pandas version >= 1.0.0 use this


# Load SQUAD Data
# Stanford Question Answering Dataset (SQuAD) 
# Replace the following strings with your own AWS credentials and bucket details
# Specify your bucket name and file name
train_file_name = 'train-v2.0.json'  # Replace with your file name

# Use the account_id to perform operations specific to the account
# For example, getting an object from a bucket
obj = s3.get_object(Bucket=bucket_name, Key=train_file_name)

# Load json data from S3
obj = s3.get_object(Bucket=bucket_name, Key=train_file_name)
data = json.loads(obj['Body'].read().decode('utf-8'))

# Normalizing data
df_squad = json_normalize(data['data'], record_path=['paragraphs', 'qas'], meta=['title'])


/tmp/ipykernel_1239/2207319538.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df_squad = json_normalize(data['data'], record_path=['paragraphs', 'qas'], meta=['title'])


In [47]:
df_squad.info()
df_squad.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   question           130319 non-null  object
 1   id                 130319 non-null  object
 2   answers            130319 non-null  object
 3   is_impossible      130319 non-null  bool  
 4   plausible_answers  43498 non-null   object
 5   title              130319 non-null  object
dtypes: bool(1), object(5)
memory usage: 5.1+ MB


,question,id,answers,is_impossible,plausible_answers,title
0,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,"[{'text': 'in the late 1990s', 'answer_start':...",False,NaN,Beyoncé
1,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,"[{'text': 'singing and dancing', 'answer_start...",False,NaN,Beyoncé
2,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,"[{'text': '2003', 'answer_start': 526}]",False,NaN,Beyoncé
3,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,"[{'text': 'Houston, Texas', 'answer_start': 166}]",False,NaN,Beyoncé
4,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,"[{'text': 'late 1990s', 'answer_start': 276}]",False,NaN,Beyoncé
